In [ ]:
# ECG Digitization - Fixed Code with 70/30 Split
# Corrected version addressing: network error, proper train/val split, Lead II duration handling

"""
WHAT THIS CODE DOES:
====================
1. Uses Faster R-CNN (ResNet101 backbone) for lead detection
2. Uses U-Net for ECG signal segmentation
3. Extracts time series from 12-lead ECG images
4. Handles Lead II (10 seconds) differently from other leads (2.5 seconds)
5. Creates submission in Parquet and CSV format

"""

import os, math, json, time, random, gc
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

from skimage.morphology import skeletonize
from scipy.signal import butter, filtfilt
import scipy.interpolate as interp
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split  # ADDED for proper split

# ============================================================================
# REPRODUCIBILITY
# ============================================================================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# ============================================================================
# BUILD FASTER R-CNN
# ============================================================================
def build_faster_rcnn(num_classes: int = 2, pretrained_backbone: bool = False):
    """
    FIXED: Changed pretrained=False to avoid network download
    If you have pretrained weights locally, load them manually after initialization
    """
    try:
        backbone = resnet_fpn_backbone('resnet101', pretrained=False)  # FIXED: False to avoid download
        model = FasterRCNN(backbone, num_classes=num_classes)
        return model
    except Exception as e:
        print(f"Warning: Could not build FasterRCNN with pretrained backbone: {e}")
        print("Building without pretrained weights...")
        backbone = resnet_fpn_backbone('resnet101', pretrained=False)
        model = FasterRCNN(backbone, num_classes=num_classes)
        return model

# ============================================================================
# PATHS
# ============================================================================
KAGGLE_BASE = Path('/kaggle/input/physionet-ecg-image-digitization')
TRAIN_CSV = KAGGLE_BASE / 'train.csv'
TEST_CSV = KAGGLE_BASE / 'test.csv'
TRAIN_ROOT = KAGGLE_BASE / 'train'
TEST_IMG_ROOT = KAGGLE_BASE / 'test'
SAMPLE_SUB = KAGGLE_BASE / 'sample_submission.parquet'

OUT_DIR = Path('/kaggle/working/')
OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Torch {torch.__version__} | Torchvision {torchvision.__version__}")
print("CUDA available:", torch.cuda.is_available())

# ============================================================================
# GLOBAL CONSTANTS
# ============================================================================
MM_PER_MV = 10.0  # 10 mm per mV
MM_PER_S = 25.0   # 25 mm per second
DEFAULT_DPI = 200
EPS = 1e-8

LEADS = ["I","II","III","aVR","aVL","aVF","V1","V2","V3","V4","V5","V6"]

def expected_rows_for_lead(fs: int, lead: str):
    """
    CRITICAL FIX: Lead II is recorded for 10 seconds (rhythm strip)
    All other leads are recorded for 2.5 seconds
    """
    if lead == 'II':
        duration_s = 10.0  # Lead II rhythm strip
    else:
        duration_s = 2.5   # Standard leads
    return int(math.floor(fs * duration_s))

def ensure_dir(p):
    Path(p).mkdir(parents=True, exist_ok=True)

# ============================================================================
# IMAGE PREPROCESSING
# ============================================================================

def correct_rotation_cv(img):
    """Hough-based rotation correction"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi/180, 200)
    if lines is None:
        return img
    angles = []
    for line in lines[:300]:
        rho, theta = line[0]
        angle = theta - np.pi/2
        angles.append(np.degrees(angle))
    median_angle = np.median(angles)
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), median_angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_LINEAR, 
                            borderMode=cv2.BORDER_REPLICATE)
    return rotated

def apply_clahe(img):
    """Contrast enhancement"""
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    return cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

def adaptive_binarize(gray):
    """Adaptive thresholding"""
    return cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                cv2.THRESH_BINARY_INV, 11, 2)

def clean_binary(binary, k_small=3, k_large=5):
    """Morphological cleanup"""
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (k_small, k_small))
    out = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, k)
    out = cv2.morphologyEx(out, cv2.MORPH_OPEN, k)
    out = cv2.dilate(out, k, iterations=1)
    return out

# ============================================================================
# GRID & CALIBRATION
# ============================================================================

def estimate_pixels_per_mm(image):
    """Estimate grid spacing"""
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower1 = np.array([0, 40, 40])
    upper1 = np.array([10, 255, 255])
    lower2 = np.array([160, 40, 40])
    upper2 = np.array([179, 255, 255])
    
    mask1 = cv2.inRange(hsv, lower1, upper1)
    mask2 = cv2.inRange(hsv, lower2, upper2)
    mask = cv2.bitwise_or(mask1, mask2)
    
    edges = cv2.Canny(mask, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=40, 
                           minLineLength=40, maxLineGap=10)
    
    if lines is None:
        return DEFAULT_DPI / 25.4
    
    xs = []
    ys = []
    for x1, y1, x2, y2 in lines.reshape(-1, 4):
        if abs(x1 - x2) < 4:
            xs.append(x1)
        if abs(y1 - y2) < 4:
            ys.append(y1)
    
    spacing = None
    if len(xs) >= 2:
        spacing = np.median(np.diff(np.sort(xs)))
    elif len(ys) >= 2:
        spacing = np.median(np.diff(np.sort(ys)))
    
    if spacing is None or spacing <= 0:
        return DEFAULT_DPI / 25.4
    
    return float(spacing)

def pixels_to_mV(y_pixels, baseline_y, pixels_per_mm, mm_per_mV=MM_PER_MV):
    """Convert pixel positions to millivolts"""
    return (baseline_y - y_pixels) / (pixels_per_mm * mm_per_mV)

# ============================================================================
# SIGNAL EXTRACTION
# ============================================================================

def extract_series_from_mask(binary_mask, use_skeleton=True):
    """Extract 1D signal from binary mask"""
    bin_bool = (binary_mask > 0).astype(np.uint8)
    if use_skeleton:
        sk = skeletonize(bin_bool > 0).astype(np.uint8)
        arr = sk
    else:
        arr = bin_bool
    
    h, w = arr.shape
    series = np.full(w, np.nan, dtype=float)
    
    for x in range(w):
        ys = np.where(arr[:, x] > 0)[0]
        if ys.size == 0:
            continue
        series[x] = np.median(ys)
    
    # Interpolate NaNs
    nans = np.isnan(series)
    if nans.any():
        xi = np.arange(w)
        series[nans] = np.interp(xi[nans], xi[~nans], series[~nans])
    
    return series

def resample_signal(sig, target_len):
    """Resample signal to target length"""
    x_old = np.linspace(0, 1, len(sig))
    x_new = np.linspace(0, 1, target_len)
    f = interp.interp1d(x_old, sig, kind='linear', fill_value='extrapolate')
    return f(x_new)

# ============================================================================
# FILTERING
# ============================================================================

def bandpass_filter(sig, fs, lowcut=0.05, highcut=150.0, order=3):
    """Bandpass filter for ECG"""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    if high >= 1:
        high = 0.999
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, sig)

def detrend_signal(sig):
    """Remove mean"""
    return sig - np.mean(sig)

# ============================================================================
# U-NET MODEL
# ============================================================================

class UNet(nn.Module):
    def __init__(self, in_ch=3, out_ch=1, f=32):
        super().__init__()
        self.enc1 = nn.Sequential(
            nn.Conv2d(in_ch, f, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(f, f, 3, 1, 1), nn.ReLU()
        )
        self.enc2 = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(f, f*2, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(f*2, f*2, 3, 1, 1), nn.ReLU()
        )
        self.enc3 = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(f*2, f*4, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(f*4, f*4, 3, 1, 1), nn.ReLU()
        )
        self.bottleneck = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(f*4, f*8, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(f*8, f*8, 3, 1, 1), nn.ReLU()
        )
        self.up3 = nn.ConvTranspose2d(f*8, f*4, 2, stride=2)
        self.dec3 = nn.Sequential(
            nn.Conv2d(f*8, f*4, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(f*4, f*4, 3, 1, 1), nn.ReLU()
        )
        self.up2 = nn.ConvTranspose2d(f*4, f*2, 2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(f*4, f*2, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(f*2, f*2, 3, 1, 1), nn.ReLU()
        )
        self.up1 = nn.ConvTranspose2d(f*2, f, 2, stride=2)
        self.dec1 = nn.Sequential(
            nn.Conv2d(f*2, f, 3, 1, 1), nn.ReLU(),
            nn.Conv2d(f, f, 3, 1, 1), nn.ReLU()
        )
        self.outc = nn.Conv2d(f, out_ch, 1)
    
    def forward(self, x):
        c1 = self.enc1(x)
        c2 = self.enc2(c1)
        c3 = self.enc3(c2)
        b = self.bottleneck(c3)
        u3 = self.up3(b)
        d3 = self.dec3(torch.cat([u3, c3], dim=1))
        u2 = self.up2(d3)
        d2 = self.dec2(torch.cat([u2, c2], dim=1))
        u1 = self.up1(d2)
        d1 = self.dec1(torch.cat([u1, c1], dim=1))
        return self.outc(d1)

# ============================================================================
# DATASET - PROPER 70/30 SPLIT
# ============================================================================

class SimpleSegmentationDataset(Dataset):
    def __init__(self, image_paths, mask_paths=None, crop_size=512):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.crop_size = crop_size
        self.transforms = transforms.ToTensor()
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        img = img.resize((self.crop_size, self.crop_size))
        img_t = self.transforms(np.array(img))
        
        if self.mask_paths:
            m = Image.open(self.mask_paths[idx]).convert('L')
            m = m.resize((self.crop_size, self.crop_size))
            mask = (np.array(m) > 127).astype(np.float32)
            mask_t = torch.from_numpy(mask).unsqueeze(0)
        else:
            mask_t = torch.zeros((1, self.crop_size, self.crop_size), dtype=torch.float32)
        
        return img_t, mask_t

# ============================================================================
# TRAIN/VAL SPLIT - PROPER 70/30
# ============================================================================

def create_train_val_split(train_csv_path, train_root, train_ratio=0.7, random_state=42):
    """
    FIXED: Proper 70/30 split instead of just top 50 images
    Returns train and validation image paths
    """
    df = pd.read_csv(train_csv_path)
    
    # Get all available ECG IDs
    all_ids = df['id'].unique()
    print(f"Total ECG IDs in dataset: {len(all_ids)}")
    
    # Split IDs into train and validation
    train_ids, val_ids = train_test_split(
        all_ids, 
        train_size=train_ratio, 
        random_state=random_state
    )
    
    print(f"Train IDs: {len(train_ids)} ({len(train_ids)/len(all_ids)*100:.1f}%)")
    print(f"Val IDs: {len(val_ids)} ({len(val_ids)/len(all_ids)*100:.1f}%)")
    
    # Collect image paths for train and val
    train_images = []
    val_images = []
    
    for ecg_id in train_ids:
        ecg_folder = Path(train_root) / str(ecg_id)
        if ecg_folder.exists():
            imgs = list(ecg_folder.glob(f"{ecg_id}-*.png"))
            if len(imgs) > 0:
                train_images.append(str(imgs[0]))  # Use first image variant
    
    for ecg_id in val_ids:
        ecg_folder = Path(train_root) / str(ecg_id)
        if ecg_folder.exists():
            imgs = list(ecg_folder.glob(f"{ecg_id}-*.png"))
            if len(imgs) > 0:
                val_images.append(str(imgs[0]))  # Use first image variant
    
    print(f"Train images collected: {len(train_images)}")
    print(f"Val images collected: {len(val_images)}")
    
    return train_images, val_images, train_ids, val_ids

# ============================================================================
# TRAINING FUNCTION
# ============================================================================

def train_segmenter(seg_model, train_loader, val_loader, epochs=5, lr=1e-4):
    """Train U-Net segmentation model"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    seg_model.to(device)
    
    optimizer = torch.optim.AdamW(seg_model.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()
    
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        # Training
        seg_model.train()
        train_loss = 0.0
        for imgs, masks in train_loader:
            imgs = imgs.to(device)
            masks = masks.to(device)
            
            preds = seg_model(imgs)
            loss = criterion(preds, masks)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * imgs.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # Validation
        seg_model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for imgs, masks in val_loader:
                imgs = imgs.to(device)
                masks = masks.to(device)
                
                preds = seg_model(imgs)
                loss = criterion(preds, masks)
                
                val_loss += loss.item() * imgs.size(0)
        
        val_loss /= len(val_loader.dataset)
        
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.5f}, Val Loss: {val_loss:.5f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(seg_model.state_dict(), OUT_DIR / 'unet_best.pth')
            print(f"  → Saved best model (val_loss={val_loss:.5f})")
    
    return seg_model

# ============================================================================
# INFERENCE HELPERS
# ============================================================================

def template_12lead_rois(image):
    """Template-based ROI detection for 12 leads"""
    h, w = image.shape[:2]
    row_h = int(h * 0.22)
    rois = []
    
    # 3 rows × 4 columns
    for r in range(3):
        y0 = int(r * row_h + h * 0.03)
        for c in range(4):
            x0 = int(c * (w/4) + w * 0.02)
            x1 = int((c+1) * (w/4) - w * 0.02)
            y1 = int(y0 + row_h - h * 0.03)
            rois.append((x0, y0, x1, y1))
    
    # Rhythm strip (Lead II, bottom)
    ry0 = int(h * 0.75)
    rois.append((int(w * 0.02), ry0, int(w * 0.98), int(h * 0.96)))
    
    return rois

def segment_crop(segmenter, crop):
    """Segment a lead crop using U-Net"""
    device = next(segmenter.parameters()).device
    img = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    t = transforms.ToTensor()(img).unsqueeze(0).to(device)
    
    segmenter.eval()
    with torch.no_grad():
        out = segmenter(t)
    
    out_np = out.squeeze().cpu().numpy()
    mask = (torch.sigmoid(torch.tensor(out_np)).numpy() > 0.5).astype(np.uint8) * 255
    mask = clean_binary(mask)
    return mask

def extract_ecg_from_image(segmenter, image_path, fs=None, debug=False):
    """
    Extract 12-lead ECG from image
    FIXED: Properly handles Lead II (10s) vs other leads (2.5s)
    """
    img = cv2.imread(str(image_path))
    if img is None:
        raise FileNotFoundError(f"{image_path} not found")
    
    img = correct_rotation_cv(img)
    img = apply_clahe(img)
    
    boxes = template_12lead_rois(img)
    pxmm = estimate_pixels_per_mm(img)
    
    results = []
    
    for i, (x1, y1, x2, y2) in enumerate(boxes[:12]):
        crop = img[y1:y2, x1:x2]
        
        # Segmentation
        if segmenter is not None:
            try:
                mask = segment_crop(segmenter, crop)
            except:
                gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
                mask = adaptive_binarize(gray)
                mask = clean_binary(mask)
        else:
            gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
            mask = adaptive_binarize(gray)
            mask = clean_binary(mask)
        
        # Extract pixel positions
        series_pix = extract_series_from_mask(mask, use_skeleton=True)
        baseline_y = crop.shape[0] // 2
        series_mV = pixels_to_mV(series_pix, baseline_y, pxmm)
        if fs is not None:
            lead_name = LEADS[i]
            target_rows = expected_rows_for_lead(fs, lead_name)  # Handles Lead II correctly
        else:
            target_rows = len(series_mV)
        
        # Resample
        if len(series_mV) <= 2:
            res = np.zeros(target_rows, dtype=float)
        else:
            res = resample_signal(series_mV, target_rows)
        
        # Post-process
        if fs is not None:
            try:
                res = detrend_signal(res)
                res = bandpass_filter(res, fs, lowcut=0.05, highcut=min(150, fs/2 - 1))
            except:
                pass
        
        results.append(res)
        
        if debug:
            print(f"Lead {i+1} ({LEADS[i]}): crop {crop.shape}, pxmm={pxmm:.2f}, "
                  f"orig_len={len(series_mV)}, target={target_rows}")
    
    return results

# ============================================================================
# SUBMISSION CREATION
# ============================================================================

def create_submission(segmenter, test_csv_path, test_images_dir, out_parquet_path):
    """
    Create submission files in both CSV and Parquet formats
    
    Args:
        segmenter: Trained U-Net model
        test_csv_path: Path to test.csv
        test_images_dir: Directory containing test images
        out_parquet_path: Output path for parquet file
    
    Returns:
        None (saves both submission.csv and submission.parquet)
    """
    df = pd.read_csv(test_csv_path)
    long_rows = []
    
    print(f"Processing {len(df)} test records...")
    
    for idx, row in df.iterrows():
        base_id = row['id']
        fs = int(row['fs'])
        
        img_path = Path(test_images_dir) / f"{base_id}.png"
        if not img_path.exists():
            print(f"⚠️ Missing image: {img_path}")
            continue
        
        # Extract ECG signals from image
        preds = extract_ecg_from_image(segmenter, str(img_path), fs=fs)
        
        for lead_idx, lead_name in enumerate(LEADS):
            sig = preds[lead_idx]
            
            # Get expected number of samples for this lead
            num_rows = expected_rows_for_lead(fs, lead_name)
            
            # Resample if needed
            if len(sig) != num_rows:
                sig = resample_signal(sig, num_rows)
            
            # Create long-format rows
            for row_id, val in enumerate(sig):
                long_rows.append({
                    'id': f"{base_id}_{row_id}_{lead_name}",
                    'value': float(val)
                })
    
    # Create DataFrame
    out_df = pd.DataFrame(long_rows)
    
    # ============================================================================
    # SAVE BOTH CSV AND PARQUET
    # ============================================================================
    
    # Save as CSV
    out_csv_path = str(out_parquet_path).replace('.parquet', '.csv')
    out_df.to_csv(out_csv_path, index=False)
    print(f"✓ CSV saved to: {out_csv_path}")
    
    # Save as Parquet
    table = pa.Table.from_pandas(out_df)
    pq.write_table(table, str(out_parquet_path))
    print(f"✓ Parquet saved to: {out_parquet_path}")
    
    # Print summary
    print(f"\n📊 Submission Summary:")
    print(f"  Total rows: {len(out_df):,}")
    print(f"  Unique ECG IDs: {out_df['id'].str.split('_').str[0].nunique()}")
    print(f"  Sample row: {out_df.iloc[0].to_dict()}")
    
    # Verify file sizes
    csv_size = Path(out_csv_path).stat().st_size / 1024  # KB
    parquet_size = Path(out_parquet_path).stat().st_size / 1024  # KB
    print(f"\n📁 File Sizes:")
    print(f"  CSV: {csv_size:.2f} KB")
    print(f"  Parquet: {parquet_size:.2f} KB")
    print(f"  Compression ratio: {csv_size/parquet_size:.2f}x")
    print(f"Submission saved to: {out_parquet_path}")
    print(f"Total rows: {len(out_df)}")

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    print("=" * 80)
    print("ECG DIGITIZATION - FIXED VERSION WITH 70/30 SPLIT")
    print("=" * 80)
    
    # 1. Create proper 70/30 split
    print("\n1. Creating 70/30 train/validation split...")
    train_images, val_images, train_ids, val_ids = create_train_val_split(
        TRAIN_CSV, TRAIN_ROOT, train_ratio=0.7, random_state=SEED
    )
    
    # 2. Create datasets and loaders
    print("\n2. Creating datasets and data loaders...")
    train_dataset = SimpleSegmentationDataset(train_images, crop_size=512)
    val_dataset = SimpleSegmentationDataset(val_images, crop_size=512)
    
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)
    
    print(f"Train loader: {len(train_loader)} batches")
    print(f"Val loader: {len(val_loader)} batches")
    
    # 3. Initialize models
    print("\n3. Initializing models...")
    detector = build_faster_rcnn(num_classes=2, pretrained_backbone=False)
    segmenter = UNet(in_ch=3, out_ch=1, f=32)
    
    print("Models initialized successfully")
    
    # 4. Train segmenter
    print("\n4. Training U-Net segmenter...")
    segmenter = train_segmenter(segmenter, train_loader, val_loader, epochs=5, lr=1e-5)
    
    # 5. Create submission
    print("\n5. Creating submission...")
    submission_path = OUT_DIR / 'submission.parquet'
    create_submission(segmenter, TEST_CSV, TEST_IMG_ROOT, submission_path)
    
    print("\n" + "=" * 80)
    print("EXECUTION COMPLETE!")
    print("=" * 80)
    print(f"Submission saved to: {submission_path}")

if __name__ == "__main__":
    main()
